# **Reporte del Modelo Baseline**
---

In [22]:
# Librerias
!pip install optuna
import os
import requests
import logging
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Funcion para crear el objeto de creación de logs
def create_logger():
  logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(levenname)s - %(message)s')
  logger = logging.getLogger('Logger')
  logger.info('Logger creado')
  return logger

In [3]:
# Función de consumo a la base de datos de Firebase para cargar el DataSet
def download_firebase(url, logger):
  logger.info("Extrayendo el archivo desde Firebase")
  df = None
  try:
    df = pd.read_csv(url)
    logger.info("Archivo cargado")
  except requests.exceptions.RequestException as e:
    logger.info(f"Error al descargar el archivo CSV: {e}")
  except pd.errors.EmptyDataError:
    logger.info("El archivo CSV está vacío.")
  except Exception as e:
    logger.info(f"Ocurrió un error inesperado: {e}")
  return df

In [4]:
# Cargar DataSet
url = 'https://firebasestorage.googleapis.com/v0/b/personalwp-8822c.appspot.com/o/diabetes_prediction_dataset.csv?alt=media&token=4d70d154-c3d0-4fa0-a3aa-9b9972dd3b95'
logger = create_logger()
df = download_firebase(url, logger)

In [5]:
# Eliminación de valores atípicos y duplicados
seventy_fifth = df['bmi'].quantile(0.75)
twenty_fifth = df['bmi'].quantile(0.25)
iqr = seventy_fifth - twenty_fifth
upper = seventy_fifth + (10 * iqr)
outliers_bmi_upper = df[(df['bmi'] > upper)]
df = pd.merge(df, outliers_bmi_upper, indicator = True, how = 'outer').query('_merge == "left_only"').drop('_merge', axis = 1)
df = df.drop_duplicates(keep = "first")

In [6]:
# Variables Categóricas a Numéricas
df['gender'] = pd.factorize(df['gender'])[0]
df['smoking_history'] = pd.factorize(df['smoking_history'])[0]

<ipython-input-6-1173de189f8b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'] = pd.factorize(df['gender'])[0]
<ipython-input-6-1173de189f8b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['smoking_history'] = pd.factorize(df['smoking_history'])[0]


## **Implementación del Modelo**

### **Partición de Datos**

In [7]:
# Separación de la 'Data' (Características)
X = df.drop(columns = 'diabetes')
X.shape

(96143, 8)

In [8]:
# Separación del 'Target' (Variable objetivo)
y = df['diabetes']
y.shape

(96143,)

In [13]:
# Partición de los datos: 70% para entrenamiento, 30% para prueba y estratificación en las etiquetas (y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [19]:
# Validación de la partición de los datos
print(f'Número de muestras en entrenamiento: {X_train.shape[0]}')
print(f'Número de muestras en prueba: {X_test.shape[0]}')
print(f'Número de características: {X_train.shape[1]}')

Número de muestras en entrenamiento: 67300
Número de muestras en prueba: 28843
Número de características: 8


In [23]:
# Distribución de la variable objetivo en los conjuntos de entrenamiento y prueba
print(f'Distribución de clases en entrenamiento: {np.bincount(y_train)}')
print(f'Distribución de clases en prueba: {np.bincount(y_test)}')

Distribución de clases en entrenamiento: [61363  5937]
Distribución de clases en prueba: [26298  2545]


### **Entrenamiento del Modelo**

In [24]:
# Función para la definición y entrenamiento de hiperparámetros a explorar
def objective(trial):
  n_neighbors = trial.suggest_int("n_neighbors", 1, 100)
  model = KNeighborsClassifier(n_neighbors = n_neighbors)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  score = accuracy_score(y_test, y_pred)
  return score

In [25]:
# Exploración de hiperparámetros
study = optuna.create_study(direction = "maximize", storage = "sqlite:///hp.db", study_name = "KNeighborsClassifier")
study.optimize(func = objective, n_trials = 100, n_jobs = -1)

[I 2024-12-11 08:27:13,774] A new study created in RDB with name: KNeighborsClassifier
[I 2024-12-11 08:27:20,069] Trial 1 finished with value: 0.9519814166348854 and parameters: {'n_neighbors': 9}. Best is trial 1 with value: 0.9519814166348854.
[I 2024-12-11 08:27:21,007] Trial 0 finished with value: 0.9495544846236522 and parameters: {'n_neighbors': 25}. Best is trial 1 with value: 0.9519814166348854.
[I 2024-12-11 08:27:29,940] Trial 2 finished with value: 0.9484450299899456 and parameters: {'n_neighbors': 33}. Best is trial 1 with value: 0.9519814166348854.
[I 2024-12-11 08:27:30,482] Trial 3 finished with value: 0.9487570641056755 and parameters: {'n_neighbors': 31}. Best is trial 1 with value: 0.9519814166348854.
[I 2024-12-11 08:27:33,822] Trial 4 finished with value: 0.9511146552023021 and parameters: {'n_neighbors': 15}. Best is trial 1 with value: 0.9519814166348854.
[I 2024-12-11 08:27:37,052] Trial 5 finished with value: 0.946815518496689 and parameters: {'n_neighbors': 69

In [32]:
# Mejor hiperparámetro
params = study.best_params
print(params)

{'n_neighbors': 9}


In [33]:
# Mejor metrica
score = study.best_value
print(score)

0.9519814166348854


**Universidad Nacional de Colombia** - *Facultad de Ingeniería*